In [1]:
import sys
from pathlib import Path
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

ROOT = Path.cwd().parent          # VisionModelsRepo/
SRC = ROOT / "src"

sys.path.insert(0, str(SRC))

import vision_model_repo as vmr
from vision_model_repo.vision_models.hf_hub import set_hf_content_environment

hf_home_path = input("Enter the path to the Hugging Face home directory: ")
hf_token = input("Enter the path to the Hugging Face token: ")
set_hf_content_environment(hf_home_path, hf_token)


In [2]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="facebook/sam-3d-objects",
    local_dir="sam_3d_checkpoints",
    local_dir_use_symlinks=False,
)

/mnt/disks/disk-1/envs/sam3d-objects/lib/python3.11/site-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 28 files:   0%|          | 0/28 [00:00<?, ?it/s]

'/home/vitimarioganiga/VisionModelsRepo/notebook/sam_3d_checkpoints'

In [3]:
from sam_3d_inference import Inference
config_path = f"sam_3d_checkpoints/checkpoints/pipeline.yaml"
inference = Inference(config_path, compile=False)

/mnt/disks/disk-1/envs/sam3d-objects/lib/python3.11/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Warp 1.11.0 initialized:
   CUDA Toolkit 12.9, Driver 12.4
   Devices:
     "cpu"      : "CPU"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /home/vitimarioganiga/.cache/warp/1.11.0


2026-01-19 16:09:53.644 | INFO     | sam3d_objects.pipeline.inference_pipeline:set_attention_backend:17 - GPU name is Tesla T4


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2026-01-19 16:09:57.352 | INFO     | sam3d_objects.model.backbone.tdfy_dit.modules.sparse:__from_env:39 - [SPARSE] Backend: spconv, Attention: sdpa
2026-01-19 16:10:03.568 | INFO     | sam3d_objects.model.backbone.tdfy_dit.modules.attention:__from_env:30 - [ATTENTION] Using backend: sdpa
/mnt/disks/disk-1/envs/sam3d-objects/lib/python3.11/site-packages/lightning/fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


[SPARSE][CONV] spconv algo: auto


/mnt/disks/disk-1/envs/sam3d-objects/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2026-01-19 16:10:06.968 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
2026-01-19 16:10:06.969 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
2026-01-19 16:10:07.058 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
/mnt/disks/disk-1/envs/sam3d-objects/lib/python3.11/site-packages/moge/model/v1.py:171: UserWarning: The following deprecated/invalid arguments are ignored: {'output_mask': True, 'split_head': True}
  warnings.warn(f"The following deprecated/invalid arguments are ignored: {deprecated_kw

In [4]:
# load image (RGBA only, mask is embedded in the alpha channel)
from sam_3d_inference import load_image, load_single_mask
image = load_image("/home/vitimarioganiga/VisionModelsRepo/sam-3d-objects/notebook/images/shutterstock_stylish_kidsroom_1640806567/image.png")
mask = load_single_mask("/home/vitimarioganiga/VisionModelsRepo/sam-3d-objects/notebook/images/shutterstock_stylish_kidsroom_1640806567", index=14)

image.shape

(4480, 6720, 3)

In [ ]:
image = image[::4,::4,:]
mask = mask[::4,::4]
print(mask.shape)

(1120, 1680)


: 

In [ ]:
# run model
output = inference(image, mask, seed=42)

# export gaussian splat
output["gs"].save_ply(f"splat.ply")
print("Your reconstruction has been saved to splat.ply")